In [17]:
from grid_search_utils import *

# This notebook is for carrying out a grid search of Beta values within a Jupyter notebook. This will certainly take longer than on a HPCC, but I've found it's more convenient to do it here since you can adjust the Beta step size.

---

In [18]:
# name of the experimental data file
# From:
# Cao, L., Coventry, B., Goreshnik, I. et al. Design of protein-binding proteins from the target structure alone. Nature 605, 551–560 (2022). https://doi.org/10.1038/s41586-022-04654-9
affinity_estimate = 'ssm_correlation_for_plotting.sc'

### Create a list of the possible combinations of conditions MDR-FEP was executed using

---

In [19]:
scorefxns = ['hardrep', 'softrep']
minimizations = ['min', 'nomin']
repacking_radii = [5, 15]
runs = []

for scorefxn in scorefxns:
    for mini in minimizations:
        for rr in repacking_radii:
            run = f'{scorefxn}__{mini}__{rr}'
            runs.append(run)

### Define the run of interest you want to perform the grid search for

In [ ]:
# put your run of interest in here.
# the format must be:
'''
[hardrep or softrep]__[nomin or min]__[5 or 15]

'''

In [ ]:
run_of_interest = 'softrep__min__5'

In [21]:
# make sure that what you requested exists by pointing it towards where your results are stored
assert os.path.exists(f'./results/all_proteins/{run_of_interest}'), 'The directory for this run does not exist'

# make sure that you have enough files in there
assert len(os.listdir(f'./results/all_proteins/{run_of_interest}')) == 10, 'You dont have data for all of the proteins in this run'

files = os.listdir(f'./results/all_proteins/{run_of_interest}')

for file in files:
    protein = file.split('__')[0]
    assert protein in protein_list, 'You dont have the protein you are looking for'

### Produce a dataframe of the MDR-FEP output contained in the 10 .npz files
#### 5 minibinders * 2 .npz files (one for the monomer and one for the dimer)

---

In [24]:
all_data = []

scorefxn = run_of_interest.split('__')[0]
minimizations = run_of_interest.split('__')[1]
repacking_radius = run_of_interest.split('__')[2]

for protein in protein_list:
        
        assert os.path.exists(f'./results/all_proteins/{run_of_interest}/{protein}__{scorefxn}__{minimizations}__{repacking_radius}__dimer.npz'), f'{protein}__{scorefxn}__{minimizations}__{repacking_radius}__dimer.npz does not exist'
            
        assert os.path.exists(f'./results/all_proteins/{run_of_interest}/{protein}__{scorefxn}__{minimizations}__{repacking_radius}__monomer.npz'), f'./results/all_proteins/{run_of_interest}/{protein}__{scorefxn}__{minimizations}__{repacking_radius}__monomer.npz does not exist'
            
        rosetta_df = parse_rosetta_data(f'./results/all_proteins/{run_of_interest}/{protein}__{scorefxn}__{minimizations}__{repacking_radius}__dimer.npz', f'./results/all_proteins/{run_of_interest}/{protein}__{scorefxn}__{minimizations}__{repacking_radius}__monomer.npz', protein)
            
        all_data.append(rosetta_df)

Processing files for bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001:
Dimer - ./results/all_proteins/softrep__min__5/bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001__softrep__min__5__dimer.npz
Monomer - ./results/all_proteins/softrep__min__5/bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001__softrep__min__5__monomer.npz
Processing files for Motif1400_ems_3hM_482_0001_7396_0001:
Dimer - ./results/all_proteins/softrep__min__5/Motif1400_ems_3hM_482_0001_7396_0001__softrep__min__5__dimer.npz
Monomer - ./results/all_proteins/softrep__min__5/Motif1400_ems_3hM_482_0001_7396_0001__softrep__min__5__monomer.npz
Processing files for ems_3hC_1642_000000001_0001:
Dimer - ./results/all_proteins/softrep__min__5/ems_3hC_1642_000000001_0001__softrep__min

In [25]:
all_rosetta_data = pd.concat(all_data)

In [26]:
all_exp_data = []

for protein in protein_list:
    
    assert os.path.exists(f'./natives/{protein}.pdb'), 'The protein {protein}.pdb does not exist or is incorrectly named'
    assert os.path.exists(f'./sequences/{protein}.seq'), 'The protein {protein}.seq does not exist or is incorrectly named'
    
    exp_df = parse_exp_data(affinity_estimate, protein, f'./natives/{protein}.pdb', f'./sequences/{protein}.seq')
    
    all_exp_data.append(exp_df)
    
all_experimental_data = pd.concat(all_exp_data)

bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001 sequence: SVKKKVRKVEKKARKAGDELAVLLARRVLEALEKGLVSEEDADESADRIEEALKK
core.import_pose.import_pose: File './natives/bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 61 76
core.conformation.Conformation: Found disulfide between residues 93 101
core.conformation.Conformation: Found disulfide between residues 127 137
Pose object created from bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001:
PDB file name: ./natives/bcov_v3_r3_ems_3hC_436_0002_000000017_0001_0001_47_64_H_.._ems_p1-15H-GBL-16H-GABBL-16H_0382_0001_0001_0001_0001_0001_0001_0001_0001.pdb
Total residues: 248
Sequence: SVKKKVR

In [27]:
all_data = combine_dfs(all_rosetta_data, all_experimental_data)

## Carry out the grid search over all Beta values.

---

```beta_step``` is the change in Beta with each step of the grid search. Small beta_step values will make the grid search slower, and vice versa. 

```metric_of_interest``` is what you're interested in finding the ideal beta for. The options are:

        correlation - Overall correlation
        correlation_intcore - Correlation in the interface core
        correlation_intbound - Correlation in the interface boundary
        correlation_moncore - Correlation in the monomer core
        correlation_monbound - Correlation in the monomer boundary
        correlation_monsurf - Correlation in the monomer boundary
        correlation_bcov - Correlation for the minibinder to IL-7Ra
        correlation_motif - Correlation for the minibinder to FGFR2
        correlation_ems - Correlation for the minibinder to VirB8
        correlation_longxing - Correlation for the minibinder to TrkA
        correlation_newr1 - Correlation for the minibinder to CD3d
        accuracy - Overall accuracy
        accuracy_intcore - Accuracy in the interface core
        accuracy_intbound - Accuracy in the interface boundary
        accuracy_moncore - Accuracy in the monomer core
        accuracy_monbound - Accuracy in the monomer boundary
        accuracy_monsurf - Accuracy in the monomer boundary
        accuracy_bcov - Accuracy for the minibinder to IL-7Ra
        accuracy_motif - Accuracy for the minibinder to FGFR2
        accuracy_ems - Accuracy for the minibinder to VirB8
        accuracy_longxing - Accuracy for the minibinder to TrkA
        accuracy_newr1 - Accuracy for the minibinder to CD3d
        
---

In [30]:
corr_df = grid_search(all_data, run_of_interest, beta_step=0.01, metric_of_interest='correlation_longxing')

Fitting (beta of 1e-09)
Length of dg_df before fitting ∆G_fold:
5738
Length of dg_df after fitting ∆G_fold:
5738
Length of to_assess before filtering: 5738
Length of to_assess after filtering: 4204
Correlation with beta 1e-09:
0.3514201674337699
Fitting (beta of 0.01)
Length of dg_df before fitting ∆G_fold:
5738
Length of dg_df after fitting ∆G_fold:
5738
Length of to_assess before filtering: 5738
Length of to_assess after filtering: 4204
Correlation with beta 0.01:
0.3488076925537703
Fitting (beta of 0.02)
Length of dg_df before fitting ∆G_fold:
5738
Length of dg_df after fitting ∆G_fold:
5738
Length of to_assess before filtering: 5738
Length of to_assess after filtering: 4204
Correlation with beta 0.02:
0.3402815963054311
Fitting (beta of 0.03)
Length of dg_df before fitting ∆G_fold:
5738
Length of dg_df after fitting ∆G_fold:
5738
Length of to_assess before filtering: 5738
Length of to_assess after filtering: 4204
Correlation with beta 0.03:
0.2657407505769872
Fitting (beta of 0.04)